#### Guía práctica Kafka

Apache Kafka es una plataforma de streaming distribuida de código abierto capaz de manejar trillones de eventos al día.
Kafka permite desacoplar los procesos. 

Tiene 3 capacidades clave:
* Publicar y suscríbirse a flujos de registros, similar a una cola de mensajes o un sistema de mensajería empresarial (esta nos permite llevar a cabo el desacoplamiento de componentes o procesos de un sistema, otorgándole la capacidad de tolerancia a fallos al sistema).
* Almacenar flujos de registros de forma duradera y tolerante a fallos (esto se debe a que la capacidad de almacenamiento de los registros depende directamente a la capacidad de almacenamiento en disco de cada uno de los nodos que formen parte del cluster kafka, entonces ¿mientras más pequeños los registros más elementos pueden ser almacenados en el cluster de kafka? Si, es aquí donde radica la importancia deconocer los formatos de serialización).
* Procesar flujos de registros a medida que se producen (Kafka está optimizado para mensajes pequeños. Mejor rendimiento con mensajes de 1KB --> utilizar un tipo de serialización y compresión adecuados puede ayudar a que nuestras aplicaciones sean más eficientes).

Kafka es una herramienta que nos permite desacoplar los flujos de datos y los sistemas.
Se usa generalmente para 2 clases de aplicaciones:
* Construir flujos de datos en tiempo real para obtener datos de manera confiable entre sistemas o aplicaciones.
* Creación de aplicaciones de streaming en tiempo real que transforman o reaccionan a los flujos de datos.

Características: 
* Distribuido de Arquitectura resistente.
* Herramienta muy tolerante a fallos.
* Escalabilidad horizontal.
* Alto rendimiento.

Casos de uso: mensajería, recopilar métricas, recopilar logs de aplicación...

Ejemplo: UBER usa Kafka para recopilar datos de usuarios, taxi y el viaje en si en tiemporeal para estimar la demanda 
y calcular el aumento de precios en tiempo real.

Conceptos de Kafka: 
Kafka se ejecuta como un clúster en uno o más servidores que pueden abarcar varios centros de datos.El clúster Kafka almacena 
flujos de registros en categorías llamadas "TOPICS". Cada registro consta de una clave, un valor y una marca de tiempo.

* Abro 2 consolas

* En las 2 consolas ejecuto: `cd kafka_2.11-2.1.0/`

* En una de las 2 consolas ejecuto: `zookeeper-server-start.sh config/zookeeper.properties`

* En la otra consola ejecuto: `kafka-server-start.sh config/server.properties`

ZOOKEEPER:
Apache kafka utiliza Zookeeper para almacenar metadatos sobre el clúster Kafka, además de gestionar los brokers 
(mantener una lista de ellos).

* Ayuda a realizar la elección del líder para las particiones.
* Envía notificaciones a kafka en caso de cambios (por ejemplo nuevo topic, brokermuere, broker se levanta, eliminación de topics, etc...).
* Es una dependencia de kafka. Kafka no puede trabajar sin zookeeper.
* Por diseño opera con un número impar de servidores (3, 5, 7).
* Zookeeper tiene un lider (maneja las escrituras) y el resto de los servidores son seguidores (manejan las lecturas).
* No almacena el offset en versiones de kafka > v0.10.

##### Topic

TOPICS: particular flujo de datos. Los topics se componen de particiones. 
* Similar a una tabla en una base de datos, es identificado por su nombre. Se pueden tener tantos topics como se desee. 
* Los topics están dividos en particiones. Cada partición es una secuencia ordenada e inmutable de registros que se agrega continuamente a un registro de confirmación estructurado. A los registros de las particiones se les asigna un número de identificación secuencial (incrementa) llamado OFFSET que identifica de manera única a cada registro dentro de la partición.
(El offset solo tiene significado para una partición en especifico. Por ejemplo offset 3 en la partición 0 no representa 
la misma data que el offset 3 de la partición 1).
* El orden está garantizado solamente dentro de una partición.
* Los datos son mantenidos solamente por un periodo de tiempo limitado (por defecto 1 semana).
* Una vez el dato ha sido escrito en una partición, este no puede ser modificado.Topics deben tener un factor de replicación > 1 (usualmente entre 2 y 3). 
* La replicación es crítica porque con esta Kafka garantiza la disponibilidad y durabilidad cuando los nodos individuales fallan inevitablemente.

BROKERS: Son los multiples servidores que componen un cluster kafka.
* Cada broker esta identificado por un ID (entero).
* Cada broker contiene cierto número de particiones que pertenecen a los topics.
* Después de conectado con cualquier broker (llamado bootstrap broker), se estará conectado al cluster en su totalidad.
* Un buen número de brokers para empezar es 3, pero algunos cluster han llegado a tener 100 brokers.

LÍDERES:
* En cualquier momento solamente 1 broker puede ser el líder para una particióndada. 
* Solamente el líder puede recibir y servir data para una partición.
* El resto de brokers sincronizarán los datos, por lo tanto cada partición tiene unlíder y multiples ISR (in-sync replica).

* Abro una tercera consola y ejecuto: `kafka-topics.sh` (muestra todos los párametros a pasar al invocar el comando y cuáles son obligatorios -REQUIRED-)

* Vamos a crear un TOPIC

* (En la tercera consola) `kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic first_topic -- create` 
  Mensaje de error: faltan las particiones

* `kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic first_topic -- create --partitions 3`
  Mensaje de error: falta el replication factor

* `kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic first_topic -- create --partitions 3 --replication-factor 2`
  Mensaje de error: el replication factor (2) es mayor al número de brokers (1)

* `kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic first_topic -- create --partitions 3 --replication-factor 1`
  --> ya se ha creado el TOPIC

* Para ver la lista con todos los topics: `kafka-topics.sh --zookeeper 127.0.0.1:2181 --list`

* Para describir el topic: `kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic first_topic --describe`

* Creo otro topic: `kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic second_topic -- create --partitions 6 --replication-factor 1`

* Listamos los topics: `kafka-topics.sh --zookeeper 127.0.0.1:2181 --list` (ahora aparece también second_topic)

* Eliminamos el second_topic: `kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic second_topic --delete`

* Vuelvo a comprobar la lista de topics: `kafka-topics.sh --zookeeper 127.0.0.1:2181 --list` (solo aparece first_topic)

##### Producer

PRODUCER:
Son aquellos encargados de escribir datos en los topics (los cuales estáncompuestos de particiones).
* Producers conocen automáticamente a cual broker y partición escribir.
* En caso de falla de un broker, los productores se recuperarán automáticamente.
* Producers pueden escoger enviar una clave con el mensaje (número, string, etc...)
	* Si la clave es null, los datos son enviados en round robin (broker 101 luegobroker 102 y después broker 103 
	  y así sucesivamente)
	* Si una clave es enviada, entonces todos los mensajes para esa clave siempre irán a la misma partición.
	* Una clave es básicamente enviada si necesitamos ordenamiento de mensajes para un campo en especifico.

* Vamos a escribir un mensaje en la consola (para salir ctrl + c):
`kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic first_topic`
`> Hola mundo`
`> Aprendiendo Kafka`
`> Más texto para seguir aprendiendo`
`> Otro mensaje`

* Pasamos más propiedades al producer, esta en especial espera la sincronización de las replicas para realizar la confirmación, lo cual garantiza que el registro no se pierda mientras al menos una replica se mantenga levantado y en ejecución, esta (acks=all) es la garantía más fuerte de disponibilidad. 

* Lanzamos el producer en la misma consola: 
`kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic first_topic --producer-property acks=all`
`> Un mensaje confirmado`
`> Otro mensaje más`
`> Seguimos aprendiendo`

* Intentamos escribir un mensaje en un topic que NO existe: `kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic new_topic`
`> Mensaje en topic nuevo inexistente`
`> Otro mensaje en este nuevo topic`
`> Un último mensaje`

* Kafka crea el topic que no existe (el topic se crea con unos valores por defecto), lo compruebo con: `kafka-topics.sh --zookeeper 127.0.0.1:2181 --list` /
`kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic new_topic --describe` 

* Vamos a ver la forma de consumir los mensajes de kafka: `kafka-console-consumer.sh`

* `kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic` --> No aparecen los mensajes que hemos escrito antes, no lee todo el topic, lee los nuevos mensajes que lleguen una vez lanzado el consumidor

* Abro una nueva consola (la cuarta) y empezamos a escribir de nuevo. Dejamos ejecutando el topic (no utilizamos ctrl+c):
`kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic first_topic`
`> Mensaje de prueba`
`> Más mensajes`

* Si quiero leer todos los mensajes desde el principio, cancelo el consumer (tercera consola) con ctrl+c y lo lanzo de nuevo:
`kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --from-beginning`

* Detenemos el consumer y el producer (ctrl+c)

##### Consumer Group

CONSUMER: Son aquellos que leen datos desde un topic (identificado por un nombre).
* Consumer saben desde cual broker leer los datos.
* En caso de fallos de un broker, los consumer saben como recuperarse.
* Los datos son leídos en orden dentro de capa partición.
* Consumers leen los datos en grupos de consumo.
	* Cada consumer dentro de un grupo lee desde unas particiones exclusivas.
	* De haber mas consumers que particiones, algunos consumer estaráninactivos.
	* Los consumer automáticamente usarán un GroupCoordinator y un ConsumerCoordinator para asignar consumers a una partición.
* Semántica de entrega para los consumidores:
	* At most once (como mucho una vez): Offsets son entregados tan pronto como los mensajes son recibidos. 
	  Si el proceso va mal, el mensaje seperderá (y no será leído de nuevo).
	* At least once (por lo menos una vez): Offsets son entregados después que el mensajes es procesado. Si el procesado 
	  va mal, el mensaje será leído de nuevo, esto puede resultar en un procesado de mensajes duplicados, por lo tanto hay 
	  que asegurarse de que el procesamiento sea idempotent.
	* Exactly once (exactamente una vez): Puede ser alcanzado por kafkautilizando kafka streams y de utilizar un 
	  sistema externo es necesario utilizarun consumidor idempotent.

CONSUMER GROUPS (grupos orientados a aplicaciones que se componen de consumidores):
* Los consumers leen los datos en grupos de consumidores.
* Cada consumer dentro de un grupo lee de una partición en especial.
* Si hay más consumers (consumidores) que particiones, entonces algunos consumers estarán inactivos.

* Ahora veremos el uso de los consumers group donde el group es el equivalente a lo que sería el nombre de nuestra aplicación. Tiene a su vez la definición de grupo ya que consiste en un grupo de consumidores. Esta ha de ser la forma de realizar el consumo de datos desde kafka por las aplicaciones(por ejemplo spark streaming) cuando involucremos alto rendimiento, concurrencia, entre otras cosas.

* En la consola donde estábamos ejecutando el consumer lanzamos el consumer que forma parte de un consumer group (tercera consola):
`kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --group my-first-application`

* En la cuarta consola lanzo de nuevo el producer para escribir el first_topic y empezar a mandar mensajes:
`kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic first_topic`
`> Hola consumidor`

* Abro una nueva consola (la quinta) y lanzo otro consumer con el mismo grupo de consumo:
`kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --group my-first-application`

* Lanzo mensaje desde la consola producer (la cuarta) y veo que los 2 consumidores van leyendo los mensajes, que se distribuyén entre los 2 consumidores
`> Mensaje 4`
`> Mensaje 5`
`> Mensaje 6`

* Puedo crear un nuevo consumer que forme parte del mismo grupo en una nueva consola (la sexta) y seguir lanzando mensajes con el producer. Los consumidores del grupo van leyendo los mensajes.

##### Ejercicios

* ¿Qué pasaría si cancelamos (utilizando Ctrl+C) uno de los consumidores (quedando 2) y seguimos enviando mensajes por el producer?
  Cancelo el consumer de la sexta consola con ctrl+c y envio más mensajes desde el producer, lo 2 consumidores que quedan siguen 
  leyendo los mensajes. 

* ¿Qué pasaría si cancelamos otro de los consumidores (quedando ya solo 1) y seguimos enviando mensajes por el producer?
  Cancelo el consumer de la quinta consola con ctrl+c, envio mensajes desde el producer y el único consumer que queda sigue 
  leyendo los mensajes. 

* ¿Qué sucede si lanzamos otro consumidor pero está vez de un grupo llamado my-second-application leyendo el topic desde el principio (--from-beginning)?
  En la consola ejecuto: 
  `kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --from-beginning --group my-second-application`
  Me aparecen todos los mensajes que he lanzado en el topic first_topic desde el principio (total 30 mensajes)

* Cancela el consumidor y ¿Qué sucede si lanzamos de nuevo el consumidor pero formando parte del grupo my-second-application?
  ¿Aparecen los mensajes desde el principio?
  `kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --group my-second-application`
  Sin poner "--from-beginning" no aparece ningún mensaje. 

* Cancela el consumer, a su vez aprovecha de enviar más mensajes utilizando el producer y de nuevo lanza el consumidor formando parte del grupo my-second_application ¿Cuál fue el resultado? 
  - Con ctrl+c cancelo el consumer de my-second-application
  - En el producer lanzo >Mensaje 20
  - El "Mensaje 20" aparece en el consumer de my-first-application
  - Vuelvo a lanzar el consumer de my-second-application:
  `kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --group my-second-application`
  - Me aparece también en el consumer de my-second-application "Mensaje 20"
  - Otro caso: si no tengo ningún consumidor activo, en una consola ejecuto el producer (`kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic first_topic`) y escribo un mensaje. Ahora abro una nueva consola y ejecuto el consumer (`kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --group my-second-application`) y directamente me aparece el mensaje que acabo de escribir en el producer.

##### Consumer Group

A los registros de las particiones se les asigna un número de identificación secuencial (incrementa) llamado OFFSET que identifica de manera única a cada registro dentro de la partición. (El offset solo tiene significado para una partición en especifico. Por ejemplo offset 3 en la partición 0 no representa la misma data que el offset 3 de la partición 1).
Con el reseteo del Offset puedo indicar que todavía no he leído nada de mis particiones. Con shift, puedo mover el Offset. Con "describe" puedo ver el offset y los mensajes que se han leído (Current-offset client-id, número de mensajes, y Log-end-offset, número de mensajes leídos). 


* Vamos a explorar el comando `kafka-consumer-groups.sh`

* Lanzamos el comando: `kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --list`
  Me aparece: my-first-application
              my-second-application

* Podemos visualizar el offset por particiones: 
  `kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --describe --group my-second-application` (o también my-first-application)

* A modo de prueba lanzamos de nuevo el consumer para el grupo my-first-application ya que puede que todavía haya mensajes que no hayamos  leído (al haber estado lanzando mensajes para ver como los consumía el grupo my-second-application):
  `kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --group my-first-application`

* Lanzamos el consumer group:
  `kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --describe --group my-first-application`

* Reseteamos el offset del consumer group:
  `kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --group my-first-application --reset-offsets --to-earliest`
  --> nos sale un mensaje de error, lo interpretamos y cambiamos la sentencia:
  `kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --group my-first-application --reset-offsets --to-earliest --execute --topic first_topic`
  --> ahora si ejecuto el comando describe, current-offset client-id es 0

* Reiniciamos el consumidor: 
  `kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --group my-first-application`
  --> Vuelven a aparecer todos los mensajes que tenía y se vuelven a procesar

* Si ahora selecciono describe ya me sale que todos los mensajes se han procesado y leído `kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --describe --group my-first-application`

* Probamos otra opción para mover el cursor. Investigar en que consiste el shift-by. 
  `kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --group my-first-application--reset-offsets --shift-by 2 --execute --topic first_topic`

* Recibimos un warning, así que vuelvo a ejecutar:
  `kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --group my-first-application--reset-offsets --shift-by -2 --execute --topic first_topic`
  Se ha modificado el offset (tiene que ser -2). Ahora new-offset ha pasado al current-offset -2 (antes 13, 10, 10 y ahora 11, 8, 8)

* Probamos de nuevo a lanzar el consumidor asociado al consumer group my-first-application y vemos los mensajes que muestra de acuerdo a la modificación que realizamos del offset: 
  `kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --groupmy-first-application` Me salen solo los últimos 6 mensajes (2 de cada partición)

##### Ejercicio final

* Crear topic llamado "topic_app" con 3 particiones y replication-factor = 1.
  `kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic topic_app --create --partitions 3 --replication-factor 1`

* Crear un producer que inserte mensajes en el topic recien creado (topic_app).
  `kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic topic_app --producer-property acks=all`

* Crear 2 consumer que forman parte de un grupo de consumo llamado "my_app".
  `kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic topic_app --group my_app` Ejecuto este comando en 2 nuevas consolas diferentes.

* Interactuar con los 3 elementos creando mensajes en el producer y visualizar como estos son consumidos por los consumer.
  --> Escribiendo mensajes en la consola del producer y los mensajes llegan y son leídos por los dos diferentes consumers. 

* Aplicar los comandos necesarios para listar los topics, grupos de consumo, así como describir cada uno de estos.
  * Para listar los topics `kafka-topics.sh --zookeeper 127.0.0.1:2181 --list` 
  * Para listar los grupos de consumo `kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --list`
  * Para describir los grupos de consumo `kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --describe my_app`

#### Guía práctica Kafka + Presto

##### 1ª parte

En esta guía daremos nuestros primeros pasos con Presto para realizar consultas sobre topics de kafka.

* Empezamos creando 4 diferentes consolas. 

* En la primera consola lanzamos zookeeper `cd kafka_2.11-2.1.0/` y `zookeeper-server-start.sh config/zookeeper.properties`

* En la segunda consola arrancamos kafka `cd kafka_2.11-2.1.0/` y `kafka-server-start.sh config/server.properties`

* En la tercera consola vamos a crear un nuevo topic: `kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic text_topic --create --partitions 3 --replication-factor 1`

* ¿Serías capaz de listar todos los topics existentes? `kafka-topics.sh --zookeeper 127.0.0.1:2181 --list` 

* En esta misma consola creamos un producer y empezamos a insertar elementos en el topic `kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic text_topic`

* ¿En otra consola serías capaz de revisar el contenido de este topic recién creado? `kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic text_topic --from-beginning`

* Empezamos a configurar Presto: `cd $HOME` y `cd presto-server-0.213/`

* Creo la carpeta catalog dentro de la carpeta etc: `mkdir etc/catalog` y creo un fichero con nano `nano etc/catalog/kafka.properties` donde escribo las propiedades (connector.name=kafka kafka.nodes=localhost:9092 kafka.table-name=text_topic kafka.hide-internal-columns=false)

* Arrancamos el servidor de Presto: `bin/launcher start`

* En otra consola lanzamos el cliente de consulta de línea de comandos de Presto: `cd $HOME` `./presto --catalog kafka --schema default`

* Listamos las tablas existentes: `SHOW TABLES;` `DESCRIBE text_topic;`

* Consultamos el topic y vemos los mensajes: `SELECT _message FROM text_topic;`

* En la consola donde tenemos el producer vamos a introducir más texto y volvemos a ejecutar el último comando en la consola con Presto para ver si se leen los nuevos mensajes.

##### 2ª parte

* Voy a crear un nuevo topic `kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic csv_topic --create --partitions 3 --replication-factor 1`

* Arrancamos el producer `kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic csv_topic` y añadimos un elemento.

* Actualizamos el fichero kafka.properties para añadir el nuevo topic (kafka.table-names=text_topic,csv_topic)

* Actualizamos el servidor de Presto (`presto-server-0.213$ bin/launcher stop` `presto-server-0.213$ bin/launcher start`)

* Ahora en la consola donde tenemos los consultas de presto podemos hacer un `SHOW TABLES;`, `DESCRIBE csv_topic;`, `SELECT_message FROM csv_topic;`

* Llegados a este punto hemos podido ver el mensaje que hemos enviado en formato csv al topic, pero lo interesante será poder consultar por el valor de alguno de los campos delimitados, por ende el siguiente paso será llevar a cabo la definición de la tabla, para ello seguiremos los siguientes pasos: detener el servidor presto, crear un nuevo fichero JSON con la estructura de la tabla, reiniciar el servidor presto para que tome los cambios.

* `bin/launcher stop`

* `mkdir etc/kafka` y creamos el fichero `nano etc/kafka/csv_topic.json` (Este fichero es la definición de la tabla donde le asignamos un nombre a cada campo delimitado)

* `bin/launcher start`

* `presto:default> DESCRIBE csv_topic;`

* `SELECT * FROM csv_topic;`

* `SELECT nombre, edad, peso, direccion FROM csv_topic WHERE edad = 38;`

##### 3ª parte

* Creo un nuevo topic `kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic json_topic --create --partitions 3 --replication-factor 1`

* Añadimos elementos al nuevo topic: >{"nombre":"JOSE ANTONIO","edad":38,"peso":75.5,"direccion":"C/ DEL LIMONERO39, PISO 1-A"}

* Edito el fichero kafka.properties usando nano y cambio kafka.table-names=text_topic,csv_topic,json_topic

* Creo un nuevo fichero JSON con la estructura de la tabla

* Reinicio el servidor Presto

* `presto:deafult> SHOW TABLES;` `presto:deafult> DESCRIBE json_topic;` `presto:default> select * from json_topic;` `presto:default> SELECT nombre, edad, peso, direccion FROMjson_topic where edad = 38;`

##### Ejercicio

Crea un nuevo topic que maneje datos con formato JSON con variedad de atributos (que tengan cadenas de caracteres, enteros, y tipos de datos reales/flotantes) inserta varios registros (al menos unos 6-10 registros) y crea una tabla en presto asociada a ese topic y lanza consultas sobre el topic (que incluso podría estar en una continua evolución, asumiendo que sigan insertando elementos) con condiciones (cláusulas WHERE) basadas en algunos de los atributos que contienen dichos JSON. NOTA: Recuerda que todo el documento json debe estar en una linea, como sugerencia utilizar un editor de texto como por ejemplo atom, utilizar el plugin Pretty JSON y aplicar minify (plasma todo un documento json en una única línea). 

* Creo un nuevo topic de prueba `kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic prueba_topic --create --partitions 3 --replication-factor 1`

* Añado registros al topic: `kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic prueba_topic`

* `bin/launcher stop`

* Edito el fichero kafka.properties y añado prueba_topic en kafka.table-names

* Creo el fichero prueba_topic.json en etc/kafka

* `SHOW TABLES;` `SELECT * FROM prueba_topic;` 

* `SELECT nombre, edad FROM prueba_topic WHERE edad = 56;` `SELECT * FROM prueba_topic WHERE peso > 60;`

* `SELECT * FROM prueba_topic WHERE edad < 30;`

##### Transmisión de datos

Kafka transporta bytes. Cuando se publican registros en un topic de Kafka, se debe especificar un serializador que pueda convertir sus objetos de datos a bytes. Kafka proporciona 3 serializadores:

* IntegerSerializer:  Cuando transmitimos únicamente datos de tipo entero.
* StringSerializer: Este es el serializador por defecto. Este como su nombre lo indica es utilizado cuando transmitimos cadenas de caracteres, por ende nos serviría para la transmisión de datos no estructurados (texto libre) y para datos en formato estructurado/semi estructurados como por ejemplo:
  * Texto delimitado (CSV, TSV)
  * JSON
  * XML
  * YAML
* ByteStreamSerializer: Utilizado para cualquier tipo de datos que estén expresados como un array de bytes, por ejemplo podría ser cualquier cadena de caracteres u objeto. 

Por último contamos con los Custom Serializer que nos permiten la serializaciónpersonalizada para objetos, entre los formatos más utilizados tenemos:

* Thrift: Fue originalmente desarrollado por Facebook. Es una bibliotecas de codificación binaria que requiere de un esquema para cualquier dato que es codificado. En Thrift se describe el esquema en el Thrift IDL (interface definition language). Thrift es un proyecto mucho más grande que Avro o Protocol Buffers(protobuf), ya que no es solo una biblioteca de serialización de datos, sino también un framework RPC completo. También tiene una cultura algo diferente: mientras que Avro y Protobuf estandarizan una única codificación binaria, Thrift abarca una gran variedad de formatos de serialización diferentes (que llama "protocolos").
* Protobuf: Fue originalmente desarrollado por Google. Al igual que Thrift es una biblioteca de codificación binaria que de igual manera requiere de un esquema. Ambos vienen con una herramienta de generación de código que toma la definición del esquema para producir clases que implementan el esquema en varios lenguajes de programación.
* Es otro formato de codificación binario que es curiosamente diferente de Protocol Buffers y Thrift. Comenzó en 2009 como un subproyecto de Hadoop, debido a que Thrift no era una buena opción para los casos de uso de Hadoop. Avro proporciona una funcionalidad similar a  Thrift, Protocol Buffers. Avro se diferencia de estos sistemas en los siguientes aspectos fundamentales:
  * Escritura dinámica: Avro no requiere que se genere código. Los datos siempre van acompañados de un esquema que permite el procesamiento completo de esos datos sin generación de código, tipos de datos estáticos,etc. Esto facilita la construcción de sistemas e idiomas genéricos de procesamiento de datos.
  * Datos sin etiquetar: dado que el esquema está presente cuando se leen los datos, es necesario codificar la información de tipo considerablemente menor con los datos, lo que resulta en un tamaño de serialización más pequeño.
  * Sin ID de campo asignados manualmente: cuando un esquema cambia, tanto el esquema antiguo como el nuevo están siempre presentes al procesar los datos, por lo que las diferencias pueden resolverse simbólicamente, utilizando nombres de campo.
  
Afortunadamente, Thrift, Protobuf y Avro soportan la evolución del esquema: puede cambiar el esquema, puede tener productores y consumidores con diferentes versiones del esquema al mismo tiempo, y todo sigue funcionando. Esa es una característica extremadamente valiosa cuando se trata de un gran sistema de producción, ya que nos permite actualizar diferentes componentes del sistema de manera independiente, en diferentes momentos, sin tener que preocuparse por la compatibilidad. Aunque en cuanto a retrocompatibilidad Thrift y Protobuf no son tan amigables como Avro ya que este resuelve esos inconvenientes sencillamente modificando (más fácilmente) los esquemas(de lectura y/o escritura). Otra ventaja que posee Avro sobre los demás es la capacidad que tiene de ser usado como un tipo de dato a consumir/producir por MapReduce además de poder ser un tipo de dato que sustente tablas en Hive (característica que comparte con Thrift).

La elección del formato a utilizar en la transmisión de los datos es un aspecto muy importante en la definición de la arquitectura y dependerá muchos factores, por lo que dependiendo de las características de proyecto la elección de un formato de texto no tiene porque ser una mala elección.

* TEXTO DELIMITADO (CSV, TSV):  Son una buena opción pero poco flexible ya que carece de un esquema por ende suele llegar a ser complicado la distinción entre formatos numéricos y cadenas de caracteres, ya que a todos los efectos lo que se estaría enviando son cadenas de caracteres.
* XML: Los ficheros XML aún gozan de cierta popularidad y sistemas no tan recientes de tipo RPC (Remote Procedure Call) aún lo utilizan como respuesta a la invocación de los servicios, por ejemplo aquellos servicios web que utilizan SOAP. Una de las desventajas de este formato es la redundancia de texto, al indicar por cada elemento una etiqueta de apertura y cierre y si hablamos de millones de elementos esto se traduce en mayor cantidad de espacio a ocupar además de contemplar la velocidad de parseo de los documentos XML, aunque por otro lado lo bueno es que se cuenta con un esquema para la validación del dato (ya sea utilizando el antiguo DTD o XML schema). 
* YAML: Resuelve el problema de la redundancia de las etiquetas obteniendo un documento más ligero que puede ser utilizado como la respuesta a devolver por frameworks RPC e incluso servicios REST. No debemos olvidar contemplar la velocidad de parseo (serialización/deserialización).
* JSON: Es un formato ampliamente utilizado en el desarrollo Web y por la gran mayoría de microservicios orientados a API via REST, con una vasta cantidad de recursos en Internet. Es un formato de serialización que al igual que YAML comparado con XML expresa la información en un documento mas pequeño. Además de contar con herramientas para la definición de schema para la validación del dato. Aplicaría alguna restricción al expresar los datos de tipo flotante por ejemplo utilizar la coma (“,”) como separador decimal.
* PROTOBUF: Puede ser una buena elección si el dato a enviar en kafka es el resultado de una llamada a un RPC como por ejemplo gRPC que utiliza este formato de serialización, además de las bondades antes explicadas.
* THRIFT: Como se ha comentado antes comparte características con Protobuf puede resultar una buena opción si el dato a enviar por kafka es el resultado de la llamadaa un RPC como por ejemplo Thrift (Si además de un formato de serialización es unRPC) o finagle.
* AVRO: Al inicio de cualquier proyecto resulta en una opción muy ventajosa para elstreaming de datos entre otras cosas por su eficiencia, soporte en gran variedad de lenguajes de programación, además de ser un formato compacto que en el caso de manejar grandes lotes de información resulta más eficiente que el JSON(por la redundancia de datos al repetir para cada documento el nombre de cada atributo), es rápido y permite la evolución del dato a través del tiempo.

##### Algoritmos de comprensión

Kafka tiene soporte para aplicar compresión a los datos por parte de los producer (en caso de que queramos aplicar algún algoritmo de compresión), los formatos de compresión son:
* GZIP
* Snappy
* LZ4

Por defecto, los mensajes se envían sin comprimir. Este parámetro puede ser configurado en snappy, GZIP o LZ4, en cuyo caso se utilizarán los algoritmos de compresión correspondientes para comprimir los datos antes de enviarlos a los brokers. Snappy fue inventado por Google para proporcionar proporciones de compresión decentes con una sobrecarga de CPU baja y un buen rendimiento, por lo que se recomienda en casos en los que tanto el rendimiento como el ancho de banda son una preocupación. La compresión GZIP normalmente utilizará más CPU y tiempo, pero resultará en mejores ratios de compresión, por lo que se recomienda en casos donde el ancho de banda de la redes más restringido al habilitar la compresión, reduce la utilización de la red y el almacenamiento, que a menudo es un cuello de botella cuando se envían mensajes a Kafka. LZ4 arroja un muy buen ratio de compresión (mayor a Snappy) y bastante rápido pero con un soporte en kafka relativamente reciente.

##### Herramientas de consulta SQL para Apache Kafka

Existen al menos 2 herramientas que permiten la consulta de registros en topics de Kafka usando SQL:

* KSQL: Es un motor de SQL de streaming de código abierto para Apache Kafka. Proporciona una interfaz SQL simple y completamente interactiva para el procesamiento de flujos en Kafka; no es necesario escribir código en un lenguajede programación como Java o Python. KSQL es de código abierto, distribuido, escalable, confiable y en tiempo real. Admite una ampliagama de potentes operaciones de procesamiento de secuencias, que incluyen agregaciones, uniones, ventanas, sesionización y mucho más.

* PRESTO: Es un motor de consultas SQL distribuidas de código abierto para ejecutar consultas analíticas interactivas contra fuentes de datos de todos los tamaños, desde gigabytes hasta petabytes. Presto fue diseñado y escrito desde cero para el análisis interactivo y se aproxima a la velocidad de los almacenes de datos comerciales al tiempo que aumenta e ltamaño de organizaciones como Facebook. Presto permite consultar datos donde se encuentran, incluidos Hive, Cassandra, bases de datos relacionales o incluso almacenes de datos propietarios. Una sola consulta de Presto puede combinar datos de múltiples fuentes, lo que permite realizar análisis en toda la organización. Presto está dirigido a analistas que esperan tiempos de respuesta que van desde menos de un segundo hasta minutos. Presto rompe la falsa elección entre tener análisis rápidos usando una solución comercial costosa o usar una solución "gratuita" lenta que requiere un hardware excesivo.

Ambas son excelentes herramientas que cumplen muy bien su cometido pero por la versatilidad de PRESTO al poder conectarse con distintas fuentes del datos se eligió comola herramienta para elaborar las consultas sobre los topics de kafka, además de que:
* En la mayoría de proyectos de Big Data, donde realmente se lidie con grandes cantidades de datos, las fuentes de datos son diversas, es muy difícil que todos los datos estén concentrados en una única fuente de datos, por ende poder contar con una herramienta que permita hacer consultas haciendo joins entre esas distintas fuentes es una ventaja y aporta versatilidad y agilidad al proyecto.
* Los proyectos de streaming con frecuencia implican el enriquecimiento de datos, donde esos provienen de kafka y son enriquecidos con otros ubicados en bases del datos relacionales, NoSQL o Hive por mencionar algunas por lo cual una herramienta como presto para depuración (debug) del proyecto de tal manera de poder experimentar el proceso de enriquecimiento del dato (proveniente desde kafka) resulta por lo menos fascinante ya que nos permite llevar a cabo pruebas de distintos escenarios, pruebas de integración y detección de errores.

##### PRESTO

Presto es un sistema distribuido que se ejecuta en un grupo de máquinas. Una instalación completa incluye un coordinador y varios trabajadores. Las consultas se envían desde un cliente como el CLI (Command Line Interface) de Presto al coordinador. El coordinador analiza y planifica la ejecución de la consulta, luego distribuye el procesamiento a los trabajadores.

Presto   es   una   herramienta   diseñada   para   consultar   de   manera   eficiente   grandes cantidades de datos mediante consultas distribuidas. Si trabaja con terabytes o petabytesde  datos,  es  probable  que  esté  utilizando  herramientas  que  interactúan  con  Hadoop  y HDFS.  Presto  se  diseñó  como  una  alternativa  a  las  herramientas  que  consultan  HDFS utilizando pipelines de trabajos de MapReduce como Hive o Pig, pero Presto no se limitaa  acceder  a  HDFS.  Presto  puede  y  se  ha  extendido  para  operar  en  diferentes  tipos  de fuentes  de  datos,  incluidas  las  bases  de  datos  relacionales  tradicionales  y  otras  fuentes de datos como Cassandra.

Presto   fue   diseñado   para   manejar   data   warehouse   y   analítica:   análisis   de   datos, agregando grandes cantidades de datos y produciendo informes. Estas cargas de trabajo a menudo se clasifican como Procesamiento analítico en línea (OLAP).

Presto posee conectores para: 
* Hadoop/Hive soportando los formatos texto, SequenceFile, RCFile, ORC, Parquet
* Cassandra
* Kudu
* Kafka
* Y muchos más 

Hay 2 tipos de SERVIDORES de Presto: 
* COORDINADOR: El  coordinador  de  Presto  es  el  servidor  que  se  encarga  de  analizar  las  declaraciones, planificar consultas y administrar los nodos de trabajo de Presto. Es el "cerebro" de una instalación  de  Presto  y  también  es  el  nodo  al  que  un  cliente  se  conecta  para  enviar declaraciones  para  su  ejecución.  Cada  instalación  de  Presto  debe  tener  un  coordinadorde Presto junto con uno o más trabajadores de Presto. Para fines de desarrollo o prueba, se puede configurar una sola instancia de Presto para realizar ambos roles. El  coordinador  realiza  un  seguimiento  de  la  actividad  de  cada  trabajador  y  coordina  la ejecución  de  una  consulta.  El  coordinador  crea  un  modelo  lógico  de  una  consulta  que involucra una serie de etapas que luego se traduce en una serie de tareas conectadas que se ejecutan en un grupo de trabajadores de Presto. Los coordinadores se comunican con los workers y clientes utilizando una API REST.
* WORKER: Un  worker  de  Presto  es  un  servidor  en  una  instalación  de  Presto  que  es responsable  de ejecutar tareas y procesar datos. Los nodos workers obtienen datos de los conectores e intercambian  datos  intermedios  entre  sí. El coordinador es responsable de obtener los resultados de los workers y devolver los resultados finales al cliente. Cuando se inicia un proceso de trabajo de Presto, se anuncia al servidor   de descubrimiento en el coordinador, lo que lo pone a disposición del coordinador de Presto para la ejecución de tareas. Los workers se comunican con otros workers y con los coordinadores de Presto medianteuna API REST.

Fuentes de datos: 

* CONECTOR: Un  conector  adapta  Presto  a  una  fuente  de  datos  como  Hive  o  una  base  de  datos relacional.  Puede  pensar  en  un  conector  de  la  misma  manera  que  piensa  en  un controlador  para  una  base  de  datos.  Es  una  implementación  del  SPI  de  Presto  que  le permite a Presto interactuar con un recurso utilizando una API estándar. Presto  contiene  varios  conectores  incorporados:  un  conector  para  JMX,  un  conector  del sistema que proporciona acceso a las tablas del sistema incorporado, un conector Hive y un conector TPCH diseñado para   servir   datos   de   referencia TPC-H. Muchos desarrolladores externos han contribuido con conectores para que Presto pueda accedera los datos en una variedad de fuentes de datos. Cada catálogo está asociado a un conector específico. Si examina un archivo de configuración del catálogo, verá  que  cada  uno  contiene  una  propiedad  obligatoria connector.name  que  utiliza  el  administrador  del  catálogo  para  crear  un  conector  para  un catálogo  determinado. Es posible que más de un catálogo  use  el  mismo  conector para acceder  a  dos  instancias  diferentes  de  una  base  de  datos  similar. Por  ejemplo, si  tiene dos grupos  Hive, puede  configurar  dos  catálogos  en  un  solo  grupo  Presto  que  usan  el conector Hive, lo que le permite consultar datos de ambos grupos (incluso dentro de la misma consulta SQL).

* CATALOGO: Un  catálogo  de  Presto  contiene  esquemas  y  hace  referencia  a  una  fuente  de  datos  a través de un conector. Por ejemplo, puede configurar un catálogo JMX para proporcionar acceso a la información JMX a través del conector JMX. Cuando ejecuta una instrucciónSQL en Presto, la está ejecutando en uno o más catálogos. Otros ejemplos de catálogos incluyen el catálogo de Hive para conectarse a una fuente de datos de Hive. Al  direccionar  una  tabla  en  Presto,  el  nombre  de  la  tabla  completamente  calificado siempre está enraizado en un catálogo. Por ejemplo, un nombre de tabla completamente calificado  de  hive.test_data.test  se  referirá  a  la  tabla  de  prueba  en  el  esquema  test_dataen el catálogo de Hive. Los catálogos se definen en los archivos de propiedades almacenados en el directorio de configuración de Presto.

* ESQUEMA: Los  esquemas  son  una  forma  de  organizar  tablas.  Juntos, un catálogo y un esquema definen un conjunto de tablas que se pueden consultar. Al acceder a Hive o una base de datos relacional como MySQL con Presto, un esquema se traduce al mismo concepto en la base de datos de destino. Otros tipos de conectores pueden optar por organizar las tablas en esquemas de una manera que tenga sentido para la fuente de datos subyacente.

* TABLA: Una tabla es un conjunto de filas no ordenadas que se organizan en columnas nombradas con tipos. Esto es lo mismo que en cualquier base de datos relacional. La asignación de datos de origen a tablas está definida por el conector.

##### Modelo de ejecución de consultas Presto

Presto ejecuta las sentencias de SQL y las convierte en consultas que se ejecutan en un grupo distribuido de coordinadores y trabajadores.

* Sentencias: Presto  ejecuta  sentencias  SQL  compatibles  con  ANSI.  Cuando  la  documentación  de Presto  se  refiere  a  una  sentencia,  se  refiere  a  declaraciones  tal  como  se  definen  en  el estándar ANSI SQL, que consta de cláusulas, expresiones y predicados.

* Consulta: Cuando  Presto  analiza  una  sentencia,  la  convierte  en  una  consulta  y  crea  un  plan  de consultas distribuidas que luego se realiza como una serie de etapas interconectadas que se  ejecutan  en  los  workers  de  Presto.  Cuando  recupera  información  sobre  una  consulta en  Presto,  recibe  un  snapshot  de  cada  componente  involucrado  en  la  producción  de  un conjunto de resultados en respuesta a una sentencia. La  diferencia  entre  una sentencia y una consulta es simple. Se puede pensar en una sentencia como el texto SQL que se pasa a Presto, mientras que una consulta se refiere a la configuración y los componentes creados para ejecutar esa sentencia. Una consulta abarca etapas, tareas,  divisiones, conectores y otros componentes y fuentes de datos que trabajan en conjunto para producir un resultado.

* Etapa: Cuando  Presto  ejecuta  una  consulta,  lo  hace  dividiendo  la  ejecución  en  una  jerarquía  de etapas. La jerarquía de etapas que comprende una consulta se parece a un árbol. Cada consulta tiene una etapa raíz que es responsable de agregar la salida de otras etapas. Las etapas son  las que usa  el coordinador para modelar un plan de consultas distribuidas, pero las etapas en sí mismas no se ejecutan en los trabajadores de Presto.

* Tarea: Para comprender cómo se ejecuta una etapa, deberá comprender que una etapa se implementa como una serie de tareas distribuidas en una red de workers de Presto. Las tareas son  el “horse  work" en la arquitectura de Presto, ya que un plan de consulta distribuida se deconstruye en una  serie  de  etapas  que  luego se traducen en tareas que luego actúan o procesan divisiones. Una tarea de Presto tiene entradas y salidas, y así como una etapa puede ejecutarse en paralelo mediante una serie de tareas, una tarea se ejecuta en paralelo con una serie de controladores.

* División: Las tareas operan en divisiones que son secciones de un conjunto de datos más grande. Las etapas en el nivel más bajo de un plan de consulta distribuida recuperan  datos  através  de divisiones  de  conectores,  y  las  etapas  intermedias  en  un  nivel  más  alto  de  un plan de consulta distribuida recuperan datos de otras etapas.

* Driver: Las tareas contienen uno o más controladores paralelos. Los controladores actúan sobre los  datos  y  combinan  los  operadores  para  producir  una  salida  que  luego  se  agrega  por una  tarea  y  luego  se  entrega  a  otra  tarea  en  otra  etapa.  Un  driver  es  una  secuencia  de instancias  de  operador,  o  puede  pensar  en  un  driver  como  un  conjunto  físico  de operadores en la memoria. Es el nivel más bajo de paralelismo en la arquitectura Presto. Un driver tiene una entrada y una salida.

* Operador: Un  operador  consume,  transforma  y  produce  datos.  Por  ejemplo,  un  escaneo  de  tabla obtiene  datos  de  un  conector  y  produce  datos  que  pueden  ser  consumidos  por  otros operadores, y un operador de filtro consume datos y produce un subconjunto al aplicar un predicado sobre los datos de entrada.

* Intercambiador: Los  intercambiadores  transfieren  datos  entre  los  nodos  de  Presto  para  diferentes  etapas de una consulta. Las tareas producen datos en un búfer de salida y consumen datos de otras tareas utilizando un cliente de intercambio.

* Línea de comando: El Presto CLI proporciona un shell interactivo basado en terminal para ejecutar consultas. La CLI es un archivo JAR de ejecución  automática, lo que significa  que  actúa como un ejecutable normal de UNIX. Una vez configurado e instalado se ejecuta de la siguiente manera:`./presto --server localhost: 8080 --catalog hive --schema default` Ejecute el CLI con la opción --help para ver las opciones disponibles. De forma  predeterminada, los resultados de las consultas se paginan utilizando el programa less, que se configura con un conjunto de opciones  cuidadosamente seleccionado. Este comportamiento se puede anular configurando la variable de entorno PRESTO_PAGER  con  el  nombre  de  un  programa  diferente, como por ejemplo more, oconfigurándolo en un valor vacío para deshabilitar completamente la paginación.

* JDBC Driver: Se puede acceder a Presto desde Java utilizando el controlador JDBC. Lo cual lo convierte en una característica muy potente  permitiendo a través de conectores la conexión desde herramientas de BI como Tableu y Qlik.

#### Spark Streaming

Spark Streaming es una extensión del Core Spark API que permite el procesamiento de flujos de datos en vivo, escalable, de alto rendimiento y tolerante a fallas. Los datos se pueden ingerir de muchas fuentes, como Kafka, Flume, Kinesis o TCP, y se pueden procesar mediante algoritmos complejos expresados con funciones de alto nivel como map, reduce, join y window. Finalmente, los datos procesados se pueden enviar a sistemas de archivos, bases de datos y paneles de control en vivo.


Internamente, funciona así: SparkStreaming recibe flujos de datos de entrada en streaming y divide los datos en lotes, que luego son procesados por el motor de Spark para generar el flujo final de resultados en lotes.

Apache Spark incluye 2 APIspara procesamiento de streams:

* DStream: Es una secuencia continua de RDDs(del mismo tipo) que representa un flujo continuo de datos. 
* Structured Streaming: Es un motor de procesamiento des treams(flujos) escalable y tolerante a fallos construido en el motor SparkSQL.

##### DSTREAM

Spark Streaming proporciona una abstracción de alto nivel llamada flujo discretizado o DStream, que representa un flujo continuo de datos. Los DStreams se pueden crear a partir de flujos de datos de entrada de fuentes como Kafka, Flume y Kinesis, o aplicando operaciones de alto nivel en otros DStreams. Internamente, un DStream se representa como una secuencia de RDD.

##### Transformaciones en un DStream
Similar a la de los RDD, las transformaciones permiten modificar los datos de la entrada DStream. DStreams admite muchas de las transformaciones disponibles en los SparkRDD normales. Ejemplos: map, filter, union, repartition, count, join...

##### Window Operations

Spark con los DStreams también proporciona cálculos en ventanas, que le permiten aplicar transformaciones en una ventana deslizante de datos. 

Cualquier operación de ventana necesita especificar 2 parámetros: 
* Longitud de la ventana: la duración de la ventana
* Intervalo de deslizamiento: el intervalo en el que se realiza la operación de la ventana

##### Structured Streaming

Structured Streaming es un motor de procesamiento de flujo escalable y tolerante a fallos construido en el motor Spark SQL. Puede expresar los cómputos sobre el streaming de la misma manera que expresaría un cálculo por lotes en datos estáticos. El motor SparkSQL se encargará de ejecutarlo de forma incremental y continua y de actualizar el resultado final a medida que continúen llegando los datos en streaming. Puede usar el Dataset / DataFrame API en Scala, Java, Python o R para expresar agregaciones sobre el stream, ventanas de eventos, combinaciones de flujo a lote (stream-to-batch), etc. El cálculo/computo se ejecuta en el mismo motor Spark SQL optimizado. Finalmente, el sistema asegura garantías de tolerancia a fallos de una sola vez de extremo a extremo através de los puntos de verificación (checkpoints) y los registros de escritura anticipada. En resumen, Structured Streaming proporciona un procesamiento rápido, escalable,tolerante a fallos, procesamiento de extremo a extremo exactly-once sin que el usuario tenga que razonar acerca stream o flujo de datos.

Structured Streaming mantiene el mismo concepto de transformaciones y acciones aplicadas a los Dataframe/Dataset y con algunas restricciones, son exactamente las mismas transformaciones. Las fuentes de entrada son:
* Kafka
* Ficheros en un sistema de ficheros distribuido como HDFS o S3
* Socket

Modo de salida: También debemos definir cómo queremos que Spark escriba los datos en esesumidero. Por ejemplo, ¿solo queremos añadir nueva información? ¿Queremos actualizarlas filas a medida que recibimos más información sobre ellas a lo largo del tiempo (por ejemplo, actualizando el número de clics para una página web determinada)? ¿Queremos que cada vez se sobrescriba completamente el conjunto de resultados (es decir, escribir siempre un archivo con el total de clics para todas las páginas)? Para hacer esto,definimos un modo de salida, similar a cómo definimos los modos de salida en la API de datos estáticos estructuradas (DataFrame/Dataset). Los modos de salida admitidos son los siguientes: 
* Append (solo agregar nuevos registros al receptor de salida)
* Update (actualizar los registros modificados en su lugar)
* Complete (reescribir la salida completa)

TRIGGERS (disparadores):
Mientras que los modos de salida definen cómo se emiten los datos, los desencadenantes o disparadores definen cuándo se emiten los datos, es decir, cuando el Structured Streaming debe verificar los nuevos datos de entrada y actualizar su resultado. De forma predeterminada, el Structured Streaming buscará nuevos registros de entrada tan pronto como haya terminado de procesar el último grupo de datos de entrada, lo que dará la menor latencia posible para los nuevos resultados. Sin embargo, este comportamiento puede llevar a escribir muchos archivos de salida pequeños cuando el receptor es un conjunto de archivos. Por lo tanto, Spark también admite desencadenantes según el tiempo de procesamiento (solo buscar nuevos datos en un intervalo fijo).

---

* Datos de tiempo de evento: Tiempo de evento significa campos de tiempo que están incrustados en sus datos (por ejemplo una fecha de alta o de modificación). Esto significa que, en lugar de procesar los datos según el tiempo que llega a su sistema, los procesa de acuerdo con el tiempo que se generaron, incluso si los registros llegan desordenados a la aplicación de streaming debido a las lentas cargas o los retrasos de la red. Expresar el procesamiento en tiempo de evento es simple en Structured Streaming. Debido a que el sistema ve los datos de entrada como una tabla, el tiempo del evento es solo otro campo en esa tabla, y su aplicación puede hacer agrupación, agregación y ventanas usando operadores de SQL estándar. Sin embargo, bajo el capó, Structured Streaming puede realizar algunas acciones especiales cuando sabe que una de sus columnas es un campo de tiempo de evento, que incluye la optimización de la ejecución de consultas o la determinación de cuándo es seguro olvidar el estado de una ventana de tiempo. Muchas de estas acciones se pueden controlar usando marcas de agua.
* Marcas de agua (watermarks): son una característica de los sistemas de streaming que le permiten especificar qué tan tarde esperan ver los datos en el momento del evento. Por ejemplo, en una aplicación que procesa registros desde dispositivos móviles, uno podría esperar que los registros lleguen hasta 30 minutos tarde debido a demoras en la carga. Los sistemas que admiten el tiempo de eventos,incluido Structured Streaming, generalmente permiten configurar marcas de agua para limitar el tiempo que necesitan para recordar datos antiguos. Las marcas de agua también se pueden usar para controlar cuándo generar un resultado para una ventana de tiempo de evento en particular (por ejemplo, esperar hasta que la marca de agua haya pasado).

#### Procesamiento de STREAMS

* El flujo de datos está presente en muchos aspectos de la vida diaria, inherente a todo lo relacionado con la transmisión de información y monitorización en tiempo real. No se limita a la transmisión de texto, sino también al audio, imagen y video. 

* El procesamiento de streams (también llamado flujo de datos) es el acto de incorporarcontinuamente nuevos datos para calcular 
un resultado. En el procesamiento de streams, los datos de entrada son ilimitados y no tienen un inicio o final predeterminado.
Simplemente forma una serie de eventos que llegan al sistema de procesamiento de flujo(por ejemplo, transacciones con tarjeta 
de crédito, clics en un sitio web o lecturas desensores de dispositivos de Internet de las cosas [IoT]).

###### Procesamiento en streaming vs procesamiento en batch 

Procesamiento por lotes (batch): el cálculo se ejecuta en un conjunto de datos de entrada fija (en el streaming son 
ilimitados). Esto puede ser un conjunto de datos a gran escala en un almacén de datos que contiene todos los eventos históricos
de una aplicación. 
El procesamiento por lotes también requiere una consulta para calcular, similar al procesamiento de streams, pero solo calcula 
el resultado una vez (mientras que en streaming se obtienen multiples versiones del resultado amedida que se está ejecutando). El stream tiene un conjunto de entrada ilimitado, el batch tiene un conjunto de entrada fijo. 

Streaming y batch a menudo necesitan trabajar juntos. 
Por ejemplo, las aplicaciones de streams a menudo necesitan unir (join) datos de entrada contra un conjunto de datos 
escrito periódicamente por un batch, y la salida de las tareas/trabajos de streaming suelen ser ficheros o tablas que 
se consultan en trabajos por lotes. Además, cualquier lógica de negocios en las aplicaciones debe funcionar de manera 
consistente a lo largo de la ejecución por streaming y batch: por ejemplo, si tiene un código personalizado para calcular 
el monto de facturación de un usuario, sería perjudicial obtener un resultado diferente cuando se ejecuta en streaming a 
cuando se ejecuta por lotes.

###### SISTEMA EN TIEMPO REAL
Los sistemas en tiempo real se clasifican como duros (hard), suaves/blandos (soft) ycercanos/casi (near). 
Puede identificar sistemas en tiempo real duros (hard) con bastante facilidad. Casi siempre se encuentran en sistemas integrados y tienen requisitos de tiempo muy estrictos que, si no se cumplen, pueden resultar en la falla total del sistema. 
* HARD: marcapasos, frenos antibloqueo. Latencia: microsegundos. Tolerancia a fallos: ninguna, posible pérdida de vida. 
* SOFT: sistemas de reserva de línea aérea. Latencia: milisegundos/segundos. Tolerancia a fallos: baja. 
* NEAR: skype video, home automation. Latencia: segundos/minutos. Tolerancia a fallos: alta. 

En muchos casos lo que tenemos es un servicio No-Hard real time con clientes que consumen datos cuando los necesitan. 
Esto se conoce como un sistema en streaming, un sistema No-Hard real time que hace que sus datos estén disponibles en el momento en que una aplicación cliente los necesita. No es ni suave ni cercano, es streaming.
El concepto de streaming de datos elimina la confusión de soft real time vs near real timey real time vs No-real time, lo que nos permite concentrarnos en diseñar sistemas que devuelvan la información que un cliente solicita en el momento en que la necesita.

Casos de uso del procesamiento en streaming: envío de notificaciones y alertas, generación de informes en tiempo real...
Aplicaciones en las que actualmente están aplicando el procesamiento en streaming: detección de intrusión, vigilancia y fraude, atención médica inteligente, analísis de deportes, procesamiento de datos geoespaciales...